In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/mink/notebooks/CameraTraps')  # append this repo to PYTHONPATH
sys.path.append('/home/mink/lib/ai4eutils')

In [3]:
import json
import os
from collections import Counter, defaultdict
from random import sample
from shutil import copyfile
from multiprocessing.pool import ThreadPool

import numpy as np
import pandas as pd
from tqdm import tqdm

import path_utils  # ai4eutils

from data_management.megadb.schema import sequences_schema_check
from data_management.megadb.converters.cct_to_megadb import process_sequences

# wps_210101

In [4]:
dataset_name = 'wps_210101'

container_root = '/mink_disk_0/camtraps/wps_210101/'  

path_to_output = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}.json' 
path_to_output_temp = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}_temp.json' 

## Step 0 - Add an entry to the `datasets` table

Done

## Step 1 - Prepare the `sequence` objects to insert into the database

No sequence info.

In [17]:
sequences = []

for folder_name in ['drive-download-20210108T171832Z-001', 'wps-1489-Images']:
    for fn in os.listdir(os.path.join(container_root, folder_name)):
        sequences.append({
            'dataset': dataset_name,
            'seq_id': f'dummy_{len(sequences)}',
            'images': [
                {
                    'file': os.path.join(folder_name, fn),
                    'frame_num': 1  # only one image, but easier for ingesting the annotations
                }
            ],
            'location': folder_name,  # no location info, so group by the folder names to be conservative
            'class': ['tbd'],  # wait for bbox annotation to fill this in
        })

In [18]:
len(sequences)

20994

In [19]:
sequences[10000]
sequences[10]

{'dataset': 'wps_210101',
 'seq_id': 'dummy_10000',
 'images': [{'file': 'wps-1489-Images/774ab70a-9d72-47a1-b484-2f2ac14e52aa.jpg',
   'frame_num': 1}],
 'location': 'wps-1489-Images',
 'class': ['tbd']}

{'dataset': 'wps_210101',
 'seq_id': 'dummy_10',
 'images': [{'file': 'drive-download-20210108T171832Z-001/512dcf85-ff7c-492d-83be-4a7b1c9e012c.jpg',
   'frame_num': 1}],
 'location': 'drive-download-20210108T171832Z-001',
 'class': ['tbd']}

## Step 2 - Pass the schema check

Once your metadata are in the MegaDB format for `sequence` items, we check that they conform to the format's schema.

If the format conforms, the following messages will be printed:

```
Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
```

For large datasets, the second step will take some time (~ a minute). 

Otherwise there will be an error message describing what's wrong. Please fix the issues until all checks are passed. You might need to write some snippets of code to loop through the `sequence` items to understand which entries have problems.

In [20]:
%%time

sequences_schema_check.sequences_schema_check(sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
CPU times: user 2.47 s, sys: 0 ns, total: 2.47 s
Wall time: 2.47 s


In [21]:
with open(path_to_output_temp, 'w', encoding='utf-8') as f:
    json.dump(sequences, f, indent=1, ensure_ascii=False)

### Step 2b - copy images to flat folder

In [22]:
def copy_file(src_path, dst_path):
    return copyfile(src_path, dst_path)

In [23]:
%%time

path_pairs = []
for seq in tqdm(sequences):
    seq_id = seq['seq_id']
    for im in seq['images']:
        src_path = os.path.join(container_root, im['file'])
        assert os.path.exists(src_path), src_path
        frame = 1
        dst_path = os.path.join('/mink_disk_0/camtraps/imerit12b', 
                                f'{dataset_name}.seq{seq_id}.frame{frame}.jpg')
        path_pairs.append((src_path, dst_path))

100%|██████████| 20994/20994 [00:00<00:00, 124511.94it/s]

CPU times: user 131 ms, sys: 44.3 ms, total: 176 ms
Wall time: 174 ms


In [24]:
len(path_pairs)  # non-empty images out of total of 19221 (38%)
path_pairs[-100]

20994

('/mink_disk_0/camtraps/wps_210101/wps-1489-Images/fd0764d0-f1b4-488e-bbf5-006fd7ad88c3.jpg',
 '/mink_disk_0/camtraps/imerit12b/wps_210101.seqdummy_20894.frame1.jpg')

In [25]:
%%time

with ThreadPool(8) as pool:
    dst_paths = pool.starmap(copy_file, path_pairs)

CPU times: user 14.4 s, sys: 27.1 s, total: 41.6 s
Wall time: 16.8 s


In [26]:
len(dst_paths)

20994